# Section 1 - Setting up the environment and loading the data.

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
# setting up pyspark
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import gdown

## Loading all the data.

In [18]:
# the required filed are downloaded fom the drive.
gdown.download_folder(id='1u2vloKJHyiK9KBHrCrPbIUU_5NBPomU3')

Retrieving folder list


Processing file 1HSaCckb7N8fLEF0GUslrXVggkYCxeETV Copy of chess_schema.png
Processing file 1PA-ldUgQECq8fIw1oYtTWTA_OqYjgjD_ Copy of chess_wc_history_game_info.csv
Processing file 1bJdq18isRDBvOizE27w4MI6oxbpEIymE Copy of chess_wc_history_moves.csv
Processing file 1ahwShbVGYrVe9TGgu05uUREO5zH2Fln1 Copy of eco_codes.csv
Building directory structure completed


Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=1HSaCckb7N8fLEF0GUslrXVggkYCxeETV
To: /content/Files-Required/Copy of chess_schema.png
100%|██████████| 295k/295k [00:00<00:00, 89.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PA-ldUgQECq8fIw1oYtTWTA_OqYjgjD_
To: /content/Files-Required/Copy of chess_wc_history_game_info.csv
100%|██████████| 596k/596k [00:00<00:00, 79.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bJdq18isRDBvOizE27w4MI6oxbpEIymE
To: /content/Files-Required/Copy of chess_wc_history_moves.csv
100%|██████████| 112M/112M [00:01<00:00, 103MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ahwShbVGYrVe9TGgu05uUREO5zH2Fln1
To: /content/Files-Required/Copy of eco_codes.csv
100%|██████████| 45.6k/45.6k [00:00<00:00, 40.9MB/s]
Download completed


['/content/Files-Required/Copy of chess_schema.png',
 '/content/Files-Required/Copy of chess_wc_history_game_info.csv',
 '/content/Files-Required/Copy of chess_wc_history_moves.csv',
 '/content/Files-Required/Copy of eco_codes.csv']

In [19]:
from pyspark.sql import SparkSession
spark = (SparkSession
 .builder
 .appName("<app_name>")
 .getOrCreate())

In [20]:
ecodf = spark.read.load("/content/Files-Required/Copy of eco_codes.csv",
                     format="csv", inferSchema="true", header="true")
movesdf = spark.read.load("/content/Files-Required/Copy of chess_wc_history_moves.csv",
                     format="csv", inferSchema="true", header="true")
game_infodf = spark.read.load("/content/Files-Required/Copy of chess_wc_history_game_info.csv",
                     format="csv", inferSchema="true", header="true")

In [21]:
ecodf.show(5)

+---+--------------------+------------------+--------+---------+
|eco|            eco_name|       eco_example|eco_type|eco_group|
+---+--------------------+------------------+--------+---------+
|A00|    Uncommon Opening|1 g4, a3, h3, etc.|       A|    Flank|
|A01|Nimzovich-Larsen ...|              1 b3|       A|    Flank|
|A02|      Bird's Opening|              1 f4|       A|    Flank|
|A03|      Bird's Opening|           1 f4 d5|       A|    Flank|
|A04|        Reti Opening|             1 Nf3|       A|    Flank|
+---+--------------------+------------------+--------+---------+
only showing top 5 rows



In [22]:
ecodf.count()

500

In [23]:
ecodf.createOrReplaceTempView('eco_codes')
movesdf.createOrReplaceTempView('history_moves')
game_infodf.createOrReplaceTempView('game_info')

In [24]:
spark.sql("show tables").show()

+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|         |    eco_codes|       true|
|         |    game_info|       true|
|         |history_moves|       true|
+---------+-------------+-----------+



# Section 2: Using pySpark for data analysis.

### 1. List of Winners of Each World champions Trophy Hint: Total Result of all rounds of Tournament for that player is considered as that player's
Score/Result.
Result attributes: winner, tournament_name


In [25]:
spark.sql("""

SELECT DISTINCT *

FROM (SELECT *, 

      max(winrate) over (partition by tournament_name) as part

      FROM (SELECT DISTINCT winner, tournament_name, count(winner) AS winrate 

            FROM game_info 

            WHERE winner != 'draw'

            GROUP BY winner, tournament_name) as t) as t1

WHERE winrate = part

ORDER BY winrate DESC

""").drop('winrate','part').show(300,truncate = False)


+---------------------+---------------+
|winner               |tournament_name|
+---------------------+---------------+
|Kasimdzhanov,R       |FideChamp2004  |
|Khalifman, Alexander |FideChamp1999  |
|Ponomariov,R         |FideChamp2002  |
|Alekhine, Alexander  |WorldChamp1929 |
|Steinitz, William    |WorldChamp1892 |
|Lasker, Emanuel      |WorldChamp1894 |
|Lasker, Emanuel      |WorldChamp1896 |
|Alekhine, Alexander  |WorldChamp1937 |
|Botvinnik, Mikhail   |WorldChamp1948 |
|Botvinnik, Mikhail   |WorldChamp1961 |
|Steinitz, William    |WorldChamp1889 |
|Anand, Viswanathan   |FideChamp1998  |
|Steinitz, William    |WorldChamp1886 |
|Euwe, Max            |WorldChamp1935 |
|Lasker, Emanuel      |WorldChamp1907 |
|Lasker, Emanuel      |WorldChamp1908 |
|Lasker, Emanuel      |WorldChamp1910b|
|Alekhine, Alexander  |WorldChamp1934 |
|Shirov,A             |FideChamp2000  |
|Anand,V              |FideChamp2000  |
|Smyslov, Vassily     |WorldChamp1954 |
|Lasker, Emanuel      |WorldChamp1909 |


### 2. List of Players with number of times they have won Tournament in descending order(Max to min).
Result attributes: player_name, number_of_wins


In [26]:
spark.sql("""

SELECT DISTINCT winner, number_of_wins

FROM (SELECT *, 

      max(number_of_wins) over (partition by tournament_name) as part

      FROM (SELECT DISTINCT winner, tournament_name, count(winner) AS number_of_wins 

            FROM game_info 

            WHERE winner != 'draw'

            GROUP BY winner, tournament_name) as t) as t1

WHERE number_of_wins = part

ORDER BY number_of_wins DESC

""").show(300,truncate = False)

+---------------------+--------------+
|winner               |number_of_wins|
+---------------------+--------------+
|Kasimdzhanov,R       |13            |
|Ponomariov,R         |12            |
|Khalifman, Alexander |12            |
|Alekhine, Alexander  |11            |
|Anand, Viswanathan   |10            |
|Alekhine, Alexander  |10            |
|Botvinnik, Mikhail   |10            |
|Lasker, Emanuel      |10            |
|Steinitz, William    |10            |
|Euwe, Max            |9             |
|Alekhine, Alexander  |8             |
|Shirov,A             |8             |
|Lasker, Emanuel      |8             |
|Anand,V              |8             |
|Botvinnik, Mikhail   |7             |
|Lasker, Emanuel      |7             |
|Smyslov, Vassily     |7             |
|Fischer, Robert James|7             |
|Karpov, Anatoly      |6             |
|Steinitz, William    |6             |
|Spassky, Boris V     |6             |
|Alekhine, Alexander  |6             |
|Kasparov, Gary       |6 

### 3. Most and Least Popular eco move in world championship history.
Result attributes: eco, eco_name, number_of_occurences
Final result will have only two rows

In [27]:
spark.sql("""

WITH t3 as (SELECT * 

            FROM (SELECT t.eco, t.eco_name, count(t.eco_name) as number_of_occurences

                 FROM (select eco_codes.eco, eco_codes.eco_name from history_moves

                        left join game_info on history_moves.game_id = game_info.game_id
      
                        left join eco_codes on game_info.eco = eco_codes.eco) as t

            GROUP BY t.eco_name, t.eco) as t2

            ORDER BY number_of_occurences desc)

(select * FROM t3 ORDER BY number_of_occurences DESC LIMIT 1)

UNION

(select * FROM t3 ORDER BY number_of_occurences ASC LIMIT 1)

""").drop('score').show(1000,truncate = False)

+---+----------------------------------------------------+--------------------+
|eco|eco_name                                            |number_of_occurences|
+---+----------------------------------------------------+--------------------+
|C42|Petrov Defense                                      |5599                |
|D54|Queen's Gambit Declined, Anti-Neo-Orthodox Variation|30                  |
+---+----------------------------------------------------+--------------------+



### 4. Find the eco move with most winnings.
Ps. Use this opening move in your next chess game🙂
Result attributes: eco, eco_name


In [28]:
spark.sql("""

SELECT *, count(eco_name) as winrate

FROM (select eco_codes.eco, eco_codes.eco_name from game_info

      left join eco_codes on game_info.eco = eco_codes.eco 

      WHERE winner != 'draw') as t

GROUP BY eco_name, eco

ORDER BY winrate desc limit 1

""").drop('winrate').show(truncate = False)


+---+-----------------+
|eco|eco_name         |
+---+-----------------+
|B90|Sicilian, Najdorf|
+---+-----------------+



### 5. Longest and shortest game ever played in a world championship in terms of move.

  Chess Funda: "move" is completed once both White and Black have played one turn. e.g If a game lasts 10 moves, both White and Black have
played 10 moves)

  Result attributes: game_id, event, tournament_name, number_of_moves
Final result will have only two rows

In [29]:
spark.sql("""

WITH t3 as (SELECT t2.game_id, event, tournament_name, part as number_of_moves

            FROM (SELECT *

                  FROM (SELECT *,

                        max(move_no_pair) over (partition by game_id) as part

                        FROM history_moves) as t

                  WHERE move_no_pair = t.part

                  ORDER BY move_no_pair DESC) as t2

           LEFT JOIN game_info on t2.game_id = game_info.game_id)

(select * FROM t3 ORDER BY number_of_moves DESC LIMIT 1)

UNION

(select * FROM t3 ORDER BY number_of_moves ASC LIMIT 1)

""").drop().show(1000,truncate = False)

+------------------------------------+-----------------------+---------------+---------------+
|game_id                             |event                  |tournament_name|number_of_moves|
+------------------------------------+-----------------------+---------------+---------------+
|4424a0a4-3732-407c-bf0c-da47f6d91921|FIDE WCh KO            |FideChamp2000  |146            |
|1846cede-0037-4f04-9dc2-bf948cf08d72|World Championship 28th|WorldChamp1972 |1              |
+------------------------------------+-----------------------+---------------+---------------+



### 6. Shortest and Longest Draw game ever Played.
Result attributes: game_id, event, tournament_name, number_of_moves
Final result will have only two rows

In [30]:
spark.sql("""

with t as (SELECT DISTINCT * 

            FROM (SELECT game_info.game_id, event, tournament_name, move_no_pair, winner,

                  max(move_no_pair) over (partition by game_info.game_id) as number_of_moves

                  FROM history_moves

                  LEFT JOIN game_info on history_moves.game_id = game_info.game_id)

            WHERE (move_no_pair = number_of_moves) and (winner = 'draw')

            ORDER BY number_of_moves DESC)

(select * FROM t ORDER BY number_of_moves DESC LIMIT 1)

UNION

(select * FROM t ORDER BY number_of_moves ASC LIMIT 1)            

""").drop('winner','move_no_pair').show(1000,truncate = False)

+------------------------------------+-------------+---------------+---------------+
|game_id                             |event        |tournament_name|number_of_moves|
+------------------------------------+-------------+---------------+---------------+
|88f34084-e4df-4908-b381-9502ca65e9bf|FIDE WCh KO  |FideChamp2002  |129            |
|a27c2b95-fb62-4b63-afc4-60bd5aced6d6|FIDE-Wch k.o.|FideChamp1998  |6              |
+------------------------------------+-------------+---------------+---------------+



### 7. Most and Least rated Player.
Result attributes: player_name, elo
Chess Funda: elo is the rating of the player in chess tournament.
Final result will have only two rows


In [31]:
spark.sql("""

with t as (SELECT *

            FROM (SELECT DISTINCT winner, winner_elo FROM game_info

                  GROUP BY winner_elo, winner

                  ORDER BY winner_elo )

            WHERE winner_elo IS NOT NULL)

(select winner as player_name, winner_elo as elo FROM t ORDER BY winner_elo DESC LIMIT 1)

UNION

(select winner as player_name, winner_elo as elo FROM t ORDER BY winner_elo ASC LIMIT 1)            

""").show(truncate = False)

+---------------+----+
|player_name    |elo |
+---------------+----+
|Carlsen, Magnus|2870|
|Ciuksyte,D     |2271|
+---------------+----+



### 8. 3rd Last Player with most Loss.
Result attributes: player_name
Final result will have only one row

In [32]:
spark.sql("""

SELECT *

FROM (SELECT *,

      ROW_NUMBER() OVER (ORDER BY no_of_loss DESC) as row_no

      FROM (SELECT loser as player_name, count(loser) as no_of_loss 

            FROM game_info

            GROUP BY loser

            ORDER BY no_of_loss DESC))

WHERE row_no = 4

""").drop('row_no','no_of_loss').show(truncate = False)


+---------------+
|player_name    |
+---------------+
|Karpov, Anatoly|
+---------------+



### 9. How many times players with low rating won matches with their total win Count.
Result attributes: player_name, win_count


In [33]:
spark.sql("""

SELECT winner as player_name, count(winner) as win_count FROM game_info 

WHERE winner_loser_elo_diff < 0

group by winner

ORDER BY win_count DESC

""").show(1000,truncate = False)

+-----------------------+---------+
|player_name            |win_count|
+-----------------------+---------+
|Karpov, Anatoly        |20       |
|Kasimdzhanov,R         |11       |
|Macieja,B              |9        |
|Kortschnoj, Viktor     |8        |
|Kramnik,V              |8        |
|Grischuk,A             |7        |
|Ponomariov,R           |7        |
|Khalifman, Alexander   |6        |
|Ehlvest,J              |5        |
|Short, Nigel D         |5        |
|Akopian, Vladimir      |5        |
|Kasparov, Gary         |5        |
|Dominguez,L            |5        |
|Sulskis,S              |5        |
|Nisipeanu, Liviu Dieter|4        |
|Ni Hua                 |4        |
|Tkachiev, Vladislav    |4        |
|Gelfand,B              |4        |
|Nakamura,H             |4        |
|Volkov,S               |4        |
|Filippov,Va            |4        |
|Shaked, Tal            |3        |
|Leitao,R               |3        |
|Milos, Gilberto        |3        |
|Anand,V                |3  

### 10. Move Sequence for Each Player in a Match.
Result attributes: game_id, player_name, move_sequence, move_count


In [34]:
spark.sql("""

SELECT game_id, player, move_sequence, move_no as move_count

FROM (SELECT game_id, player, move_sequence, move_no, move_no_pair,

      max(move_no_pair) over (partition by game_id) as part

      FROM history_moves)

WHERE move_no_pair = part

""").show(truncate = False)

+------------------------------------+----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|game_id                             |player                |move_sequence                                                                                                                                                                                                                                                             

### 11. Total Number of games where losing player has more Captured score than Winning player.
Hint: Captured score is cumulative, i.e., for 3rd capture it will have score for 1, 2, and 3rd.
Result attributes: total_number_of_games Final result will have only one row

In [35]:
spark.sql("""

with ldf as (SELECT *

              FROM (SELECT *,

                    max(move_no_pair) over (partition by game_id) as part

                    FROM history_moves)

              WHERE move_no_pair = part)


SELECT count(*) as games_losing_player_has_more_Captured_score

FROM (SELECT *,

      CASE

          WHEN (color = 'White') AND csw > csb THEN '1'

          WHEN (color = 'Black') AND csb > csw THEN '1'

          ELSE '0'

      END AS tf

      FROM (SELECT captured_score_for_white as csw, captured_score_for_black as csb, color

            FROM ldf

            LEFT JOIN game_info on ldf.game_id = game_info.game_id

            WHERE player = loser))

WHERE tf = '1'

""").show(1000,truncate = False)


+-------------------------------------------+
|games_losing_player_has_more_Captured_score|
+-------------------------------------------+
|85                                         |
+-------------------------------------------+



# CONCLUTION

### The above questions required proper level of understanding on the 3 tables that we got. being new to SQL lot of experience related to conditional Statements in SQL, joins, sub queries and also some complex WITH statements were learned and put to use. The most important part was all the questions were answred with the expectation set right. we also used union and we were in a situvation were OFFSET did not work and we used window function row_number() to solve the problem.